# CA7: Deep Q-Networks (DQN) and Value-Based Methods
## Deep Reinforcement Learning - Session 7

### Course Information
- **Course**: Deep Reinforcement Learning
- **Session**: 7
- **Topic**: Deep Q-Networks (DQN) and Advanced Value-Based Methods
- **Focus**: Complete theoretical foundations and practical implementations

### Learning Objectives

By the end of this notebook, you will understand:

1. **Theoretical Foundations**:
   - Q-learning and its limitations in complex environments
   - Deep Q-Networks (DQN) architecture and training process
   - Experience replay and target networks
   - Double DQN and addressing overestimation bias
   - Dueling DQN and advantage decomposition

2. **Implementation Skills**:
   - Complete DQN implementation from scratch
   - Experience replay buffer design and management
   - Target network updates and stability techniques
   - Advanced variants: Double DQN, Dueling DQN
   - Performance analysis and debugging techniques

3. **Practical Applications**:
   - Training DQN on classic control environments
   - Hyperparameter tuning and optimization strategies
   - Comparison with different DQN variants

### Project Structure

This notebook uses a modular implementation organized as follows:

```
CA7/
├── dqn/                    # Core DQN implementations
│   ├── core.py            # Basic DQN, ReplayBuffer, DQNAgent
│   ├── double_dqn.py      # Double DQN implementation
│   ├── dueling_dqn.py     # Dueling DQN architecture
│   └── utils.py           # Visualization and analysis utilities
├── experiments/           # Experiment scripts
│   ├── basic_dqn_experiment.py
│   └── comprehensive_dqn_analysis.py
├── requirements.txt       # Python dependencies
└── CA7.ipynb             # This educational notebook
```

### Contents Overview

1. **Section 1**: Theoretical Foundations of Deep Q-Learning
2. **Section 2**: Basic DQN Implementation and Core Concepts
3. **Section 3**: Experience Replay and Target Networks
4. **Section 4**: Double DQN and Overestimation Bias
5. **Section 5**: Dueling DQN and Value Decomposition
6. **Section 6**: Performance Analysis and Comparisons

In [ ]:
# Essential Setup and Imports

import sys
import os
sys.path.append(os.path.dirname(os.path.abspath('.')))

# Import our modular DQN implementations
from dqn import (
    DQNAgent, DoubleDQNAgent, DuelingDQNAgent,
    QNetworkVisualization, PerformanceAnalyzer
)

# Standard libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import gymnasium as gym
import warnings

# Set random seeds for reproducibility
np.random.seed(42)

# Matplotlib settings
plt.style.use('seaborn-v0_8')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12
warnings.filterwarnings('ignore')

print("="*60)
print("CA7: Deep Q-Networks (DQN) and Value-Based Methods")
print("="*60)
print("Modular implementation loaded successfully!")
print("Available agents: DQNAgent, DoubleDQNAgent, DuelingDQNAgent")
print("="*60)

# Section 1: Theoretical Foundations of Deep Q-Learning

## 1.1 From Tabular Q-Learning to Deep Q-Networks

Traditional Q-learning works well for discrete, small state spaces where we can maintain a Q-table. However, in complex environments like Atari games or continuous control tasks, the state space becomes enormous, making tabular methods impractical.

### The Q-Learning Foundation

The Q-learning update rule is:
```
Q(s, a) ← Q(s, a) + α[r + γ max Q(s', a') - Q(s, a)]
                                a'
```

Where:
- `Q(s, a)`: Action-value function
- `α`: Learning rate
- `r`: Reward
- `γ`: Discount factor
- `s'`: Next state

### The Deep Q-Network Approach

DQN replaces the Q-table with a deep neural network `Q(s, a; θ)` that approximates the Q-values for all actions given a state. The network parameters `θ` are updated to minimize the temporal difference (TD) error.

## 1.2 Core Challenges in Deep Q-Learning

### 1. Instability and Divergence
- Neural networks can be unstable when used with bootstrapping
- Updates can cause the target to change rapidly
- Non-stationary target problem

### 2. Correlation in Sequential Data
- RL data is highly correlated (sequential states)
- Violates the i.i.d. assumption of supervised learning
- Can lead to poor generalization

### 3. Overestimation Bias
- Max operator in Q-learning can lead to overestimation
- Amplified in function approximation settings
- Can cause instability and poor performance

## 1.3 DQN Innovations

### Experience Replay
- Store experiences in a replay buffer
- Sample random batches for training
- Breaks correlation and improves sample efficiency

### Target Network
- Use a separate target network for computing targets
- Update target network periodically
- Provides stability during training

### Mathematical Formulation

The DQN loss function is:
```
L(θ) = E[(r + γ max Q(s', a'; θ⁻) - Q(s, a; θ))²]
                   a'
```

Where `θ⁻` represents the parameters of the target network.

In [ ]:
# Theoretical Foundations Visualization

# Create visualization instance
visualizer = QNetworkVisualization()

print("1. Visualizing Core Q-Learning Concepts...")
visualizer.visualize_q_learning_concepts()

print("\n2. Demonstrating Overestimation Bias...")
visualizer.demonstrate_overestimation_bias()

# Section 2: Basic DQN Implementation and Core Concepts

## 2.1 Deep Q-Network Architecture

The DQN architecture typically consists of:

1. **Input Layer**: Processes the state representation
2. **Hidden Layers**: Fully connected layers for feature extraction
3. **Output Layer**: Outputs Q-values for all possible actions

### Key Design Principles:

- **State Preprocessing**: Normalize inputs for stable training
- **Network Depth**: Balance between expressiveness and training stability
- **Activation Functions**: ReLU is commonly used for hidden layers
- **Output Layer**: Linear activation for Q-value regression

## 2.2 Experience Replay Buffer

The replay buffer serves several critical functions:

1. **Decorrelation**: Breaks temporal correlations in sequential data
2. **Sample Efficiency**: Allows multiple updates from the same experience
3. **Stability**: Provides more stable gradients through diverse batches

## 2.3 Training Loop and Key Components

The DQN training process involves:

1. **Action Selection**: ε-greedy exploration strategy
2. **Environment Interaction**: Execute actions and collect experiences
3. **Experience Storage**: Add experiences to replay buffer
4. **Network Updates**: Sample batches and perform gradient descent
5. **Target Network Updates**: Periodic synchronization for stability

Let's demonstrate the basic DQN implementation.

In [ ]:
# Basic DQN Demonstration

# Create environment
env = gym.make('CartPole-v1')
state_dim = env.observation_space.shape[0]
action_dim = env.action_space.n

print(f"Environment: CartPole-v1")
print(f"State dimension: {state_dim}")
print(f"Action dimension: {action_dim}")
print(f"Goal: Balance pole for as long as possible (max 500 steps)")
print()

# Create DQN agent
agent = DQNAgent(
    state_dim=state_dim,
    action_dim=action_dim,
    lr=1e-3,                    # Learning rate
    gamma=0.99,                 # Discount factor
    epsilon_start=1.0,          # Initial exploration
    epsilon_end=0.01,           # Final exploration
    epsilon_decay=0.995,        # Exploration decay
    buffer_size=10000,          # Experience replay buffer size
    batch_size=64,              # Training batch size
    target_update_freq=100      # Target network update frequency
)

# Training parameters
num_episodes = 100
max_steps_per_episode = 500

print("Training Configuration:")
print(f"  Episodes: {num_episodes}")
print(f"  Max steps per episode: {max_steps_per_episode}")
print(f"  Learning rate: {agent.optimizer.param_groups[0]['lr']}")
print(f"  Gamma: {agent.gamma}")
print(f"  Epsilon decay: {agent.epsilon_decay}")
print()

# Quick training demonstration
print("Starting training demonstration...")
print("-" * 40)

episode_rewards = []
for episode in range(num_episodes):
    reward, steps = agent.train_episode(env, max_steps=max_steps_per_episode)
    episode_rewards.append(reward)

    if (episode + 1) % 25 == 0:
        avg_reward = np.mean(episode_rewards[-25:])
        print(f"Episode {episode+1:3d} | Avg Reward: {avg_reward:6.1f} | Epsilon: {agent.epsilon:.3f}")

print("-" * 40)
print("Training demonstration completed!")
print()

# Final evaluation
print("Final Evaluation:")
eval_results = agent.evaluate(env, num_episodes=10)
print(f"Mean Reward: {eval_results['mean_reward']:.2f} ± {eval_results['std_reward']:.2f}")

env.close()

# Section 3: Experience Replay and Target Networks

## 3.1 Experience Replay: Breaking the Correlation Chain

Experience replay is one of the most crucial innovations in DQN. It addresses several fundamental challenges:

### Problems with Sequential Training
1. **Temporal Correlation**: Consecutive states are highly correlated
2. **Non-stationarity**: The data distribution changes as the policy evolves
3. **Sample Inefficiency**: Each experience is used only once

### Benefits of Experience Replay
1. **Decorrelation**: Random sampling breaks temporal dependencies
2. **Sample Efficiency**: Multiple learning updates from each experience
3. **Stability**: More stable gradients from diverse batches

## 3.2 Target Networks: Stabilizing the Moving Target

The target network addresses the moving target problem in Q-learning:

### The Problem
In standard Q-learning, both the predicted Q-value and the target Q-value are computed using the same network, creating instability.

### The Solution
- Maintain two networks: main (online) and target
- Use target network to compute stable targets
- Update target network less frequently than main network

Let's analyze the impact of these components.

In [ ]:
# Experience Replay and Target Network Analysis

# Create analyzer
analyzer = PerformanceAnalyzer()

# Analyze Q-value distributions
print("Analyzing Q-value distributions...")
agent, _ = analyzer.analyze_q_value_distributions(
    agent, gym.make('CartPole-v1'), num_samples=500
)

# Section 4: Double DQN and Overestimation Bias

## 4.1 The Overestimation Problem in Q-Learning

Standard Q-learning suffers from a systematic overestimation bias due to the max operator in the Bellman equation. This problem is amplified in function approximation settings.

### Mathematical Analysis of Overestimation Bias

In standard DQN, the target is computed as:
```
y = r + γ max Q(s', a'; θ⁻)
          a'
```

The issue arises because we use the same network to both **select** the action and **evaluate** it.

### Impact on Learning
- **Suboptimal Policies**: Overestimated Q-values can lead to poor action selection
- **Instability**: Inconsistent value estimates cause training instability
- **Slow Convergence**: Biased estimates slow down learning

## 4.2 Double DQN Solution

Double DQN addresses this by **decoupling action selection from action evaluation**:

### Key Insight
Use the main network to select actions, but the target network to evaluate them:

```
y = r + γ Q(s', argmax Q(s', a'; θ), θ⁻)
              a'
```

### Benefits
- **Reduced Bias**: Eliminates the correlation between selection and evaluation
- **Better Stability**: More consistent Q-value estimates
- **Improved Performance**: Often leads to better final policies

Let's compare standard DQN vs Double DQN.

In [ ]:
# Double DQN Demonstration

# Create environment
env = gym.make('CartPole-v1')
state_dim = env.observation_space.shape[0]
action_dim = env.action_space.n

# Test both variants
agents = {
    'Standard DQN': DQNAgent(
        state_dim=state_dim, action_dim=action_dim,
        lr=1e-3, epsilon_decay=0.995, buffer_size=10000
    ),
    'Double DQN': DoubleDQNAgent(
        state_dim=state_dim, action_dim=action_dim,
        lr=1e-3, epsilon_decay=0.995, buffer_size=10000
    )
}

print("Comparing Standard DQN vs Double DQN...")
print("=" * 50)

results = {}
num_episodes = 50

for name, agent in agents.items():
    print(f"\nTraining {name}...")
    episode_rewards = []

    for episode in range(num_episodes):
        reward, _ = agent.train_episode(env, max_steps=500)
        episode_rewards.append(reward)

        if (episode + 1) % 25 == 0:
            avg_reward = np.mean(episode_rewards[-25:])
            print(f"  Episode {episode+1}: Avg Reward = {avg_reward:.1f}")

    # Evaluate
    eval_results = agent.evaluate(env, num_episodes=10)
    results[name] = {
        'rewards': episode_rewards,
        'eval_performance': eval_results,
        'final_performance': np.mean(episode_rewards[-10:])
    }

# Visualize comparison
PerformanceAnalyzer.plot_learning_curves(results)

env.close()
print("Comparison completed!")

# Section 5: Dueling DQN and Value Decomposition

## 5.1 The Motivation Behind Dueling Architecture

Standard DQN learns Q-values directly, but these can be decomposed into two meaningful components:

### Value Decomposition Theory
The Q-function can be decomposed as:
```
Q(s,a) = V(s) + A(s,a)
```

Where:
- **V(s)**: State value function - "How good is this state?"
- **A(s,a)**: Advantage function - "How much better is action a compared to average?"

### Why This Matters

1. **State Value Learning**: Many states have similar values regardless of action
2. **Action Ranking**: What matters most is the relative advantage of actions
3. **Sample Efficiency**: Decoupling allows better generalization
4. **Faster Learning**: State values can be learned from all experiences

## 5.2 Dueling Network Architecture

### Network Structure
```
Input State
     |
Feature Extraction
     |
   Split into two streams
     /              \
Value Stream    Advantage Stream
   V(s)           A(s,a)
     \              /
      Combining Module
           |
        Q(s,a)
```

### Combining the Streams

To address identifiability issues, we subtract the mean advantage:
```
Q(s,a) = V(s) + A(s,a) - (1/|A|) Σ A(s,a')
                                    a'
```

## 5.3 Benefits of Dueling Architecture

1. **Better Value Estimation**: State values learned more efficiently
2. **Improved Policy**: Better action selection through advantage learning
3. **Robustness**: More stable learning across different environments

Let's demonstrate the Dueling DQN architecture.

In [ ]:
# Dueling DQN Demonstration

# Create environment
env = gym.make('CartPole-v1')
state_dim = env.observation_space.shape[0]
action_dim = env.action_space.n

# Test different variants
agents = {
    'Standard DQN': DQNAgent(
        state_dim=state_dim, action_dim=action_dim,
        lr=1e-3, epsilon_decay=0.995, buffer_size=10000
    ),
    'Dueling DQN (Mean)': DuelingDQNAgent(
        state_dim=state_dim, action_dim=action_dim,
        dueling_type='mean', lr=1e-3, epsilon_decay=0.995, buffer_size=10000
    ),
    'Dueling DQN (Max)': DuelingDQNAgent(
        state_dim=state_dim, action_dim=action_dim,
        dueling_type='max', lr=1e-3, epsilon_decay=0.995, buffer_size=10000
    )
}

print("Comparing DQN variants with Dueling architecture...")
print("=" * 55)

results = {}
num_episodes = 60

for name, agent in agents.items():
    print(f"\nTraining {name}...")
    episode_rewards = []

    for episode in range(num_episodes):
        reward, _ = agent.train_episode(env, max_steps=500)
        episode_rewards.append(reward)

        if (episode + 1) % 30 == 0:
            avg_reward = np.mean(episode_rewards[-30:])
            print(f"  Episode {episode+1}: Avg Reward = {avg_reward:.1f}")

    # Evaluate
    eval_results = agent.evaluate(env, num_episodes=10)
    results[name] = {
        'rewards': episode_rewards,
        'eval_performance': eval_results,
        'final_performance': np.mean(episode_rewards[-10:])
    }

# Visualize comparison
PerformanceAnalyzer.plot_learning_curves(results)

# Analyze value-advantage decomposition for dueling agents
print("\nAnalyzing Value-Advantage Decomposition...")
sample_state = [0.1, 0.1, 0.1, 0.1]  # Example CartPole state

for name, data in results.items():
    agent = data.get('agent')
    if hasattr(agent, 'get_value_advantage_decomposition'):
        decomp = agent.get_value_advantage_decomposition(sample_state)
        print(f"\n{name} decomposition for sample state:")
        print(f"  Q-values: {decomp['q_values']}")
        print(f"  State value: {decomp['value']:.3f}")
        print(f"  Advantages: {decomp['advantage']}")

env.close()
print("\nDueling DQN demonstration completed!")

# Section 6: Performance Analysis and Comparisons

## 6.1 Comprehensive Performance Analysis

Let's run a comprehensive analysis comparing all DQN variants and analyze their performance characteristics.

## 6.2 Key Insights and Best Practices

### DQN Best Practices:
1. **Experience Replay**: Essential for breaking correlations and improving stability
2. **Target Networks**: Critical for preventing divergence and ensuring stable learning
3. **Double DQN**: Reduces overestimation bias, especially important in complex environments
4. **Dueling Architecture**: Improves value estimation and can accelerate learning

### Hyperparameter Tuning:
- **Learning Rate**: 1e-3 to 1e-4 typically works well
- **Gamma**: 0.99 for most environments
- **Epsilon Decay**: 0.995 provides good exploration-exploitation balance
- **Buffer Size**: 10,000-50,000 experiences usually sufficient
- **Batch Size**: 32-128 samples per update
- **Target Update Frequency**: 100-1000 steps

### When to Use Each Variant:
- **Basic DQN**: Good starting point, works well on simple environments
- **Double DQN**: Better for environments with overestimation issues
- **Dueling DQN**: Excellent for environments where value estimation matters
- **Combined (Double + Dueling)**: Best overall performance on complex tasks

## 6.3 Running Full Experiments

For comprehensive experiments, use the experiment scripts in the `experiments/` directory:

```bash
# Run basic DQN experiment
python experiments/basic_dqn_experiment.py

# Run comprehensive analysis (all variants)
python experiments/comprehensive_dqn_analysis.py
```

These scripts will provide detailed training curves, performance metrics, and analysis.

In [ ]:
# Final Comprehensive Analysis

print("="*60)
print("CA7 Final Analysis: DQN Variants Comparison")
print("="*60)

# Create environment for analysis
env = gym.make('CartPole-v1')
state_dim = env.observation_space.shape[0]
action_dim = env.action_space.n

# Test all variants
variants = {
    'Basic DQN': DQNAgent(
        state_dim=state_dim, action_dim=action_dim,
        lr=1e-3, epsilon_decay=0.995, buffer_size=15000
    ),
    'Double DQN': DoubleDQNAgent(
        state_dim=state_dim, action_dim=action_dim,
        lr=1e-3, epsilon_decay=0.995, buffer_size=15000
    ),
    'Dueling DQN': DuelingDQNAgent(
        state_dim=state_dim, action_dim=action_dim,
        dueling_type='mean', lr=1e-3, epsilon_decay=0.995, buffer_size=15000
    )
}

print("Training all DQN variants for comparison...")
print("-" * 50)

final_results = {}
num_episodes = 80

for name, agent in variants.items():
    print(f"\nTraining {name}...")
    episode_rewards = []

    for episode in range(num_episodes):
        reward, _ = agent.train_episode(env, max_steps=500)
        episode_rewards.append(reward)

        if (episode + 1) % 40 == 0:
            avg_reward = np.mean(episode_rewards[-40:])
            print(f"  Episode {episode+1}: Avg Reward = {avg_reward:.1f}")

    # Final evaluation
    eval_results = agent.evaluate(env, num_episodes=15)
    final_results[name] = {
        'agent': agent,
        'rewards': episode_rewards,
        'losses': agent.losses,
        'epsilon_history': agent.epsilon_history,
        'eval_performance': eval_results,
        'final_performance': np.mean(episode_rewards[-15:])
    }

# Create comprehensive visualization
PerformanceAnalyzer.plot_learning_curves(final_results)

# Performance summary
print("\n" + "="*60)
print("PERFORMANCE SUMMARY")
print("="*60)

for name, data in final_results.items():
    eval_perf = data['eval_performance']
    final_perf = data['final_performance']
    print(f"{name}:")
    print(f"  Training Performance: {final_perf:.1f}")
    print(f"  Evaluation Performance: {eval_perf['mean_reward']:.1f} ± {eval_perf['std_reward']:.1f}")
    print()

print("Key Insights:")
print("• All DQN variants can solve CartPole-v1 (avg reward > 195)")
print("• Double DQN reduces overestimation bias")
print("• Dueling DQN improves value estimation efficiency")
print("• Experience replay and target networks are essential for stability")

env.close()

print("\n" + "="*60)
print("CA7 Analysis Complete!")
print("For more detailed experiments, run:")
print("  python experiments/comprehensive_dqn_analysis.py")
print("="*60)